### 코드 순서
- [0️⃣ Initialize notebook variables](#0)
- [🧪 Azure OpenAI를 직접 호출하여 On Your Data에서 응답을 리턴하는지 테스트](#requests)
- [🧪 API Management를 직접 HTTP를 호출하여 API 테스트](#requests)

### 사전 준비 사항
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- 리소스 그룹 명, Azure OpenAI에서 배포한 deployment 이름 (예: gpt-4o), OpenAI API 버전 (예:2024-02-01)을 입력합니다.
- API Gateway URL과 Subscription Key를 입력합니다.
- Subscription Key는 Azure Portal의 API Management 리소스에서 APIs->Subscriptions->Built-in all-access subscription의 Primary key를 입력합니다.

In [ ]:
import os
import json
import datetime
import requests

resource_group_name = "{your-resource-group-name}" # 이전 단계에서 생성한 리소스 그룹 이름을 입력합니다.
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"
openai_model_name = "gpt-4o"
apim_resource_gateway_url = "{your-apim-gateway-url}" # 이전 단계에서 생성한 APIM 리소스의 게이트웨이 URL을 입력합니다.
apim_subscription_key = "{your-api-subscription-key}" # 이전 단계에서 생성한 APIM 리소스의 구독 키를 입력합니다.


<a id='requests'></a>
### 🧪 Azure OpenAI를 직접 호출하여 On Your Data에서 응답을 리턴하는지 테스트

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = "{your-azure-openai-endpoint}"
api_key = "{your0azure-openai-api-key}"
deployment = "gpt-4o"

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": "서울교대부초에 원서 접수를 할 수 있는 조건은 무엇인가요?",
        },
    ],
    extra_body={
        "data_sources":[
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": "{your-azure-ai-search-endpoint}",
                    "index_name": "school-index",
                    "authentication": {
                        "type": "api_key",
                        "key": "{your-azure-ai-search-key}",
                    }
                }
            }
        ],
    }
)

print(completion.model_dump_json(indent=2))

<a id='requests'></a>
### 🧪 API Management를 직접 HTTP를 호출하여 API 테스트

In [ ]:
url = "{your-api-url}" #예: https://apim-jl-nov26.azure-api.net/openai-s/deployments/gpt-4o/chat/completions?api-version=2024-02-01
api_runs = []
for i in range(1):
    extra_body={
            "data_sources":[
                {
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": "your-azure-ai-search-endpoint",
                        "index_name": "school-index",
                        "authentication": {
                            "type": "api_key",
                            "key": "{your-azure-ai-search-key}",
                        }
                    }
                }
            ]
        }
    
    messages={
        "messages":[
            {"role": "system", "content": "You are an AI assistant that helps people find information."},
            {"role": "user", "content": "서울교대부초와 중대부초 중에 원서 접수가 먼저 마감되는 곳은 어디인가요?"}
        ]
    }

    json_messages = messages | extra_body

    response = requests.post(url, headers = {'Ocp-Apim-Subscription-Key': '{your-apim-subscription-key}', 'work-type': 'work'}, json = json_messages)

    if (response.status_code == 200):
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "✅")
        data = json.loads(response.text)
        total_tokens = data.get("usage").get("total_tokens")
        print("💬 ", data.get("choices")[0].get("message").get("content"))
    else:
        print("▶️ Run: ", i+1, "status code: ", response.status_code, "⛔")
        print(response.text)
        total_tokens = 0
    
    print("x-ms-region:", '\x1b[1;31m'+response.headers.get("x-ms-region")+'\x1b[0m') # 이 헤더를 통하여 어느 Region의 Azure OpenAI 서비스에서 응답을 리턴했는지 파악할 수 있음
    
    api_runs.append((total_tokens, response.status_code))
